In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
else:
    print(f'Found GPU at: {device_name}')

# Alternative way to check
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")


2025-05-18 16:47:01.069525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747586821.274210      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747586821.338685      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


I0000 00:00:1747586834.684535      35 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [2]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 16.0 MB/s eta 0:00:0000:01
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [3]:
!pip install transformers

# **PART 2**

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

In [5]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset
from torch.optim import AdamW
import pandas as pd

# Suppress transformer warnings
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# Load tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
from datasets import load_dataset

# Load the Tamil NER dataset
dataset = load_dataset("google/xtreme", "PAN-X.hi")
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/38.9k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/39.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
# Preprocessing the dataset to align NER tags with tokens
label_list = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)  # Ignore these tokens in the loss
        elif word_id != previous_word_id:
            new_labels.append(labels[word_id])  # Only label first token of the word
        else:
            # Continue with I- tags for inner tokens, otherwise ignore
            new_labels.append(labels[word_id] if label_list[labels[word_id]].startswith("I-") else -100)
        previous_word_id = word_id
    return new_labels


def preprocess_data(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)
    all_word_ids = [tokenized_inputs.word_ids(i) for i in range(len(batch["tokens"]))]

    # Align labels with word ids for each example in the batch
    all_aligned_labels = [
        align_labels_with_tokens(labels, word_ids) for labels, word_ids in zip(batch["ner_tags"], all_word_ids)
    ]

    # Add aligned labels to tokenized inputs
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs


train_dataset = train_data.map(preprocess_data, batched=True)
validation_dataset = validation_data.map(preprocess_data, batched=True)
test_dataset = test_data.map(preprocess_data, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
# Load model
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model with more capacity for multilingual tasks
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=dataset['train'].features['ner_tags'].feature.num_classes)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [8]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2c08fd6653c54fef529ea88976968a651c14bca61bcde5a8be0820f3c70cf4b7
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [9]:
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# Define compute_metrics function
def compute_metrics(pred):
    label_ids = pred.label_ids
    predictions = pred.predictions.argmax(-1)
    
    # Remove ignored index (special tokens)
    true_labels = [[label_list[label_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                   for i in range(len(label_ids))]
    true_predictions = [[label_list[pred_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                        for i in range(len(predictions))]
    
    # Calculate metrics using seqeval
    precision = precision_score(true_labels, true_predictions)
    recall = recall_score(true_labels, true_predictions)
    f1 = f1_score(true_labels, true_predictions)
    accuracy = accuracy_score(true_labels, true_predictions)
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [10]:
# A-LLRD Optimizer (fixed for mBERT and checkpointing)
def get_optimizer(model, base_lr=2e-5, min_decay=0.85, max_decay=0.95):
    def compute_gradient_norms(model):
        norms = []
        for i, layer in enumerate(model.bert.encoder.layer):
            norm = 0
            for param in layer.parameters():
                if param.grad is not None:
                    norm += param.grad.norm().item() ** 2
            norms.append(np.sqrt(norm) if norm > 0 else 1e-8)
        return norms

    class ALLRDOptimizer(AdamW):
        def __init__(self, params, model, base_lr, min_decay, max_decay, betas=(0.9, 0.999), eps=1e-8, amsgrad=False, weight_decay=0.01, fused=False, capturable=False, differentiable=False, maximize=False, foreach=None):
            defaults = dict(
                lr=base_lr,
                betas=betas,
                eps=eps,
                amsgrad=amsgrad,
                weight_decay=weight_decay,
                fused=fused,
                capturable=capturable,
                differentiable=differentiable,
                maximize=maximize,
                foreach=foreach
            )
            super().__init__(params, **defaults)
            self.model = model
            self.base_lr = base_lr
            self.min_decay = min_decay
            self.max_decay = max_decay
            self.step_count = 0

        def step(self, closure=None):
            self.step_count += 1
            norms = compute_gradient_norms(self.model)
            norms = [min(max(norm, 1e-8), 5.0) for norm in norms]
            norms = torch.tensor(norms, device=next(self.model.parameters()).device)
            decay_factors = torch.softmax(-norms, dim=0).cpu().numpy()
            decay_factors = self.min_decay + (self.max_decay - self.min_decay) * decay_factors / decay_factors.max()
            if self.step_count % 100 == 0:
                print(f"Step {self.step_count}: Decay factors: {decay_factors.tolist()}")
            # Update learning rates of existing param_groups instead of rebuilding
            for i, param_group in enumerate(self.param_groups[:-1]):  # Exclude classifier group
                param_group['lr'] = self.base_lr * decay_factors[i]
            self.param_groups[-1]['lr'] = self.base_lr  # Classifier LR remains fixed
            super().step(closure)

    # Initialize parameter groups (only once)
    optimizer_grouped_parameters = [
        {
            "params": layer.parameters(),
            "lr": base_lr,
            "betas": (0.9, 0.999),
            "eps": 1e-8,
            "amsgrad": False,
            "weight_decay": 0.01,
            "fused": False,
            "capturable": False,
            "differentiable": False,
            "maximize": False,
            "foreach": None
        } for layer in model.bert.encoder.layer
    ] + [{
        "params": model.classifier.parameters(),
        "lr": base_lr,
        "betas": (0.9, 0.999),
        "eps": 1e-8,
        "amsgrad": False,
        "weight_decay": 0.01,
        "fused": False,
        "capturable": False,
        "differentiable": False,
        "maximize": False,
        "foreach": None
    }]
    return ALLRDOptimizer(optimizer_grouped_parameters, model, base_lr, min_decay, max_decay)

In [11]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated to eval_strategy as requested
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.05,
    report_to="none",
    logging_steps=100,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    save_total_limit=1
)

# Initialize trainer (without early stopping)
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(get_optimizer(model, base_lr=2e-5, min_decay=0.85, max_decay=0.95), None)
)

# Train the model
trainer.train()
metrics = trainer.evaluate(test_dataset)
metrics_df = pd.DataFrame([metrics])
print(metrics_df)

/tmp/ipykernel_35/772242240.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.333900,0.355009,0.903282,0.764394,0.823002,0.792616
2,0.232300,0.371076,0.921945,0.823344,0.851550,0.837209
3,0.180300,0.364727,0.923921,0.837967,0.860522,0.849095
4,0.121300,0.354350,0.932814,0.838407,0.876020,0.856801
5,0.115400,0.371606,0.928862,0.843014,0.876020,0.859200
6,0.095800,0.419018,0.937864,0.861526,0.893148,0.877052
7,0.048700,0.462914,0.936436,0.848319,0.884992,0.866267
8,0.054800,0.460130,0.935668,0.869496,0.885808,0.877576
9,0.057000,0.512040,0.933472,0.837481,0.878467,0.857484
10,0.052800,0.581738,0.927764,0.838088,0.886623,0.861673


Step 100: Decay factors: [0.9340299409756851, 0.932309099540142, 0.9306834394643586, 0.9291071064912609, 0.9298333175512996, 0.9307268443516861, 0.9353605272679242, 0.9371322435531596, 0.9428315180565613, 0.9462301784457954, 0.95, 0.9485193935473505]
Step 200: Decay factors: [0.926731282232205, 0.9268078401511505, 0.9247788578505612, 0.9198709396278915, 0.9228360569825484, 0.9300960842193496, 0.9383578033843936, 0.9413637712046179, 0.9449361987931639, 0.9470242932466738, 0.949787800568482, 0.95]
Step 300: Decay factors: [0.9276398079388611, 0.9234409672565788, 0.9251313187519442, 0.9224905900516837, 0.9195256904899113, 0.9278108801582632, 0.9396046765664261, 0.9431742735787951, 0.9461513268347677, 0.9481990993654981, 0.95, 0.9495826623204859]
Step 400: Decay factors: [0.9300093132099168, 0.9290074641598434, 0.9258054105875435, 0.9217449950242331, 0.9203900115660814, 0.9256716177021678, 0.9358547891814945, 0.9415593904991907, 0.945163214914215, 0.9476270513488961, 0.9498306936199467, 0.

   eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1  \
0   0.393574        0.94669         0.88348     0.901466  0.892382   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  
0        4.4153                  226.487                 28.311   10.0  


In [12]:
# Inspect the log history to understand the structure of logs
for log in trainer.state.log_history:
    print(log)


{'loss': 0.9126, 'grad_norm': 16.164682388305664, 'learning_rate': 1.96832e-05, 'epoch': 0.16, 'step': 100}
{'loss': 0.4698, 'grad_norm': 15.103615760803223, 'learning_rate': 1.93632e-05, 'epoch': 0.32, 'step': 200}
{'loss': 0.4229, 'grad_norm': 35.75712585449219, 'learning_rate': 1.90432e-05, 'epoch': 0.48, 'step': 300}
{'loss': 0.4027, 'grad_norm': 21.412677764892578, 'learning_rate': 1.87232e-05, 'epoch': 0.64, 'step': 400}
{'loss': 0.382, 'grad_norm': 0.582518994808197, 'learning_rate': 1.84032e-05, 'epoch': 0.8, 'step': 500}
{'loss': 0.3339, 'grad_norm': 3.9227747917175293, 'learning_rate': 1.80832e-05, 'epoch': 0.96, 'step': 600}
{'eval_loss': 0.35500937700271606, 'eval_accuracy': 0.9032824678889011, 'eval_precision': 0.7643939393939394, 'eval_recall': 0.8230016313213703, 'eval_f1': 0.7926158680282797, 'eval_runtime': 4.3896, 'eval_samples_per_second': 227.811, 'eval_steps_per_second': 28.476, 'epoch': 1.0, 'step': 625}
{'loss': 0.2681, 'grad_norm': 0.7551491260528564, 'learning_

In [13]:
# Log validation metrics per epoch
print("\nValidation Metrics per Epoch:")
validation_metrics = []
for log in trainer.state.log_history:
    if "eval_f1" in log:
        epoch_metrics = {
            "epoch": log.get("epoch", None),
            "eval_loss": log.get("eval_loss", None),
            "eval_accuracy": log.get("eval_accuracy", None),
            "eval_precision": log.get("eval_precision", None),
            "eval_recall": log.get("eval_recall", None),
            "eval_f1": log.get("eval_f1", None)
        }
        validation_metrics.append(epoch_metrics)
validation_df = pd.DataFrame(validation_metrics)
print(validation_df)

# Evaluate on test dataset (overall metrics)
print("\nTest Metrics:")
test_metrics = trainer.evaluate(test_dataset)
test_metrics_summary = {
    "test_accuracy": test_metrics["eval_accuracy"],
    "test_precision": test_metrics["eval_precision"],
    "test_recall": test_metrics["eval_recall"],
    "test_f1": test_metrics["eval_f1"]
}
test_metrics_df = pd.DataFrame([test_metrics_summary])
print(test_metrics_df)

# Compute per-class metrics manually
print("\nPer-Class Test Metrics:")
model.eval()
all_true_labels = []
all_pred_labels = []

for batch in trainer.get_test_dataloader(test_dataset):
    batch = {k: v.to(model.device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1).cpu().numpy()
    labels = batch["labels"].cpu().numpy()
    for i in range(len(labels)):
        true_labels = [label_list[l] for l, p in zip(labels[i], predictions[i]) if l != -100]
        pred_labels = [label_list[p] for l, p in zip(labels[i], predictions[i]) if l != -100]
        all_true_labels.extend(true_labels)
        all_pred_labels.extend(pred_labels)

class_report = classification_report([all_true_labels], [all_pred_labels], output_dict=True)
per_class_data = []
for label, metrics in class_report.items():
    if label in label_list:
        per_class_data.append({
            "label": label,
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1-score": metrics["f1-score"],
            "support": metrics["support"]
        })
per_class_df = pd.DataFrame(per_class_data)
print(per_class_df)


Validation Metrics per Epoch:
    epoch  eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1
0     1.0   0.355009       0.903282        0.764394     0.823002  0.792616
1     2.0   0.371076       0.921945        0.823344     0.851550  0.837209
2     3.0   0.364727       0.923921        0.837967     0.860522  0.849095
3     4.0   0.354350       0.932814        0.838407     0.876020  0.856801
4     5.0   0.371606       0.928862        0.843014     0.876020  0.859200
5     6.0   0.419018       0.937864        0.861526     0.893148  0.877052
6     7.0   0.462914       0.936436        0.848319     0.884992  0.866267
7     8.0   0.460130       0.935668        0.869496     0.885808  0.877576
8     9.0   0.512040       0.933472        0.837481     0.878467  0.857484
9    10.0   0.581738       0.927764        0.838088     0.886623  0.861673
10   10.0   0.393574       0.946690        0.883480     0.901466  0.892382

Test Metrics:
   test_accuracy  test_precision  test_recall   test_f

In [15]:
# Test model on example sentences
def test_model(sentences):
    model.eval()
    for sentence in sentences:
        tokenized_input = tokenizer(sentence, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        input_ids = tokenized_input["input_ids"].to(model.device)
        attention_mask = tokenized_input["attention_mask"].to(model.device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = np.argmax(outputs.logits.cpu().numpy(), axis=2)
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
        predicted_labels = [label_list[p] for p in predictions[0]]
        print(f"\nTesting sentence: {sentence}")
        for token, label in zip(tokens, predicted_labels):
            if token not in tokenizer.special_tokens_map.values() and token.strip():
                print(f"Word: '{token}', Predicted Label: {label}")

hindi_test_sentences = [
    "मेरा नाम अमित शर्मा है और मैं दिल्ली से हूँ।",
    "यह एक नई तकनीकी कंपनी, टाटा टेक्नोलॉजीज, की शुरुआत है।",
    "हिंदी भाषा भारत की एक प्रमुख भाषा है।",
    "यह पौधा भारत, चीन, जापान और माइक्रोनेशिया जैसे देशों में पाया जाता है।",
    "सलमान खान, दीपिका पादुकोण, रणवीर सिंह, और प्रियंका चोपड़ा बॉलीवुड के प्रसिद्ध अभिनेता हैं।"
]
test_model(hindi_test_sentences)


Testing sentence: मेरा नाम अमित शर्मा है और मैं दिल्ली से हूँ।
Word: 'मे', Predicted Label: O
Word: '##रा', Predicted Label: O
Word: 'नाम', Predicted Label: O
Word: 'अ', Predicted Label: B-PER
Word: '##म', Predicted Label: B-PER
Word: '##ित', Predicted Label: I-PER
Word: 'श', Predicted Label: I-PER
Word: '##र्मा', Predicted Label: I-PER
Word: 'है', Predicted Label: O
Word: 'और', Predicted Label: O
Word: 'म', Predicted Label: O
Word: '##ैं', Predicted Label: O
Word: 'दिल्ली', Predicted Label: B-LOC
Word: 'से', Predicted Label: O
Word: 'ह', Predicted Label: O
Word: '##ू', Predicted Label: O
Word: '##ँ', Predicted Label: O
Word: '।', Predicted Label: O

Testing sentence: यह एक नई तकनीकी कंपनी, टाटा टेक्नोलॉजीज, की शुरुआत है।
Word: 'यह', Predicted Label: O
Word: 'एक', Predicted Label: O
Word: 'नई', Predicted Label: O
Word: 'तक', Predicted Label: B-ORG
Word: '##नी', Predicted Label: I-ORG
Word: '##की', Predicted Label: I-ORG
Word: 'कंपनी', Predicted Label: O
Word: ',', Predicted Label: O
W